# Homework

## Q1. Install MLflow
To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 2.13.0


## Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from [here](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script preprocess_data.py located in the folder [homework](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/cohorts/2024/02-experiment-tracking/homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:
```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER?`

In [2]:
!ls

Untitled.ipynb	artifacts   dataset  output		 train.py
__pycache__	backend.db  mlruns   preprocess_data.py


In [3]:
!python preprocess_data.py --raw_data_path dataset --dest_path ./output

In [4]:
import glob

In [5]:
glob.glob("output/*")

['output/val.pkl', 'output/test.pkl', 'output/dv.pkl', 'output/train.pkl']

In [6]:
len(glob.glob("output/*"))

4

## Q3. Train a model with autolog
We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [homework](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/cohorts/2024/02-experiment-tracking/homework).

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with `mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the min_samples_split parameter:

* 2
* 4
* 8
* 10

In [7]:
from train import *

In [8]:
run_train(data_path="output")

2024/06/03 16:28:18 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/06/03 16:28:19 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/06/03 16:28:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
mlflow.get_tracking_uri()

'http://127.0.0.1:5000'

In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/1', creation_time=1717432889759, experiment_id='1', last_update_time=1717432889759, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/0', creation_time=1717431317777, experiment_id='0', last_update_time=1717431317777, lifecycle_stage='active', name='Default', tags={}>]

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336

In [5]:
!python hpo.py

  7%|▋          | 1/15 [00:09<02:17,  9.83s/trial, best loss: 5.370086069268862]/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

 13%|█▍         | 2/15 [00:11<01:07,  5.20s/trial, best loss: 5.370086069268862]/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.p

In [6]:
ls

Untitled.ipynb  artifacts/  dataset/  mlruns/  preprocess_data.py
__pycache__/    backend.db  hpo.py    output/  train.py


In [22]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/2', creation_time=1717434831461, experiment_id='2', last_update_time=1717434831461, lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/1', creation_time=1717432889759, experiment_id='1', last_update_time=1717432889759, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/0', creation_time=1717431317777, experiment_id='0', last_update_time=1717431317777, lifecycle_stage='active', name='Default', tags={}>]

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [9]:
client.search_registered_models()

[]

In [20]:
client.search_runs(experiment_ids=["1"])

[<Run: data=<RunData: metrics={'rmse': 5.355041749098929}, params={'max_depth': '20',
  'min_samples_leaf': '1',
  'min_samples_split': '9',
  'n_estimators': '19',
  'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "6b4d9a1f168b4230a9fe31455f402ad5", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2024-06-03 16:56:07.637900", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.9.19", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.5.0", '
                              '"serialization_format": "cloudpickle"

In [32]:
client.search_runs(experiment_ids=["1"], order_by=["metrics.test_rmse ASC"])[0].info.run_id

'6b4d9a1f168b4230a9fe31455f402ad5'

In [17]:
ls

Untitled.ipynb  artifacts/  dataset/  mlruns/  preprocess_data.py  train.py
__pycache__/    backend.db  hpo.py    output/  register_model.py


In [31]:
!python register_model.py

2024/06/03 17:37:31 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/06/03 17:37:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/06/03 17:37:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
2024/06/03 17:37:39 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/06/03 17:37:45 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/06

In [23]:
EXPERIMENT_NAME = "random-forest-best-models"
client.get_experiment_by_name(EXPERIMENT_NAME)

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/2', creation_time=1717434831461, experiment_id='2', last_update_time=1717434831461, lifecycle_stage='active', name='random-forest-best-models', tags={}>

In [30]:
client.search_experiments(filter_string=f"name = '{EXPERIMENT_NAME}'")[0].experiment_id

'2'